<h1>HMM & CRF</h1>

# 0. HMM

线性回归、逻辑回归假设输入样本的变量（特征）之间是相互独立的，朴素贝叶斯假设特征之间是条件独立的，在现实中，提前到的特征之间往往有千丝万缕的联系，按照这些朴素的假设建模会损失很多信息。比如图像（按照像素表示）的相邻像素之间比较相似，一篇文章相邻的单词、句子、段落之间也有一定的时间序列上的关联性。

为了建模变量之间的关联性，衍生了概率图模型。

按照顶点之间的边是否有向，概率图模型分为有向概率图模型（如贝叶斯网络）和无向概率图模型（如条件随机场）。如果顶点之间关联的边特别多（如全连通图），会给计算带来特别大的麻烦。所以为了降低计算复杂度，会尽量选择结合物理意义和物理假设，建立条件独立关系、最大子团关系等来减少不必要的连通边。

## 0.0 HMM定义

隐马尔科夫模型(Hidden Markov Models)建模时间序列，假设有隐含变量按照一定的概率来生成特征变量，并且隐含变量之间也以一定的概率相互转移。

$N$个输出节点，$N$个隐含节点的HMM包含隐含状态$K$个隐含状态$z_n=s_k$, $M$个输出状态$x_n=o_m$；

> 隐含状态的初始概率分布为$\pi$；

> 隐含状态之间的状态转移概率矩阵为$A_{ij} = p(z_{n}=s_j|z_{n-1} = s_i)$；

> 隐含状态到输出之间的输出观测概率矩阵为$B_{ij} = p(o_{n} = x_j|z_{n} = s_i)$。


完全数据(X, Z)的联合概率分布为，

> $p(X, Z|\lambda) = p(z_1)p(x_1|z_1) \prod_{n=2}^N p(x_n|z_n) \prod_{n=2}^N p(z_n|z_{n-1})$

## 0.1 HMM问题

+ Likelihood

> 给定HMM模型$\lambda=[\pi, A, B]$和观测序列$X = \{x_1, x_2, ..., x_n\}$，求该序列的概率$p(X|\lambda)$;

+ Decode

> 给定HMM模型$\lambda=[\pi, A, B]$和观测序列$X = \{x_1, x_2, ..., x_n\}$，给出最有可能的隐含状态序列$Z = \{z_1, z_2, ..., z_n\}$;

+ Learning

> 给定观测序列$X = \{x_1, x_2, ..., x_n\}$，如何调整模型参数$\lambda = [\pi, A, B]$，使得该序列出现的概率$p(X|\lambda)$最大？


## 0.2 Likelihood问题求解（forward, backward）

> $p(X|\lambda) = \sum_{z}^Z p(X, z|\lambda)$





+ forward

令 $\alpha_{t,k} = p(x_1, x_2, ..., x_t, z_t = s_k|\lambda)$，输入是$x_1, x_2, ..., x_t$，当前隐含状态是状态$k$, $z_t = s_k$，

> $\alpha_{1,k} = p(x_1, z_1 = s_k|\lambda) = \pi_k b_{k,x_1}$

> $\alpha_{t+1,k} = p(x_1, x_2, ..., x_t, x_{t+1}, z_{t+1} = s_k|\lambda) $

> $ = \sum_{i=1}^K p(x_1, x_2, ..., x_t, z_t = s_i, x_{t+1}, z_{t+1} = s_k; \lambda)$

> $ = \sum_{i=1}^K p(x_1, x_2, ..., x_t|z_t = s_i, x_{t+1}, z_{t+1} = s_k;\lambda) * p(z_t=s_i, x_{t+1}, z_{t+1}=s_k)$

> $ = \sum_{i=1}^K p(x_1, x_2, ..., x_t|z_t = s_i; \lambda) * p(x_{t+1}, z_{t+1} = s_k | z_t = s_i) p(z_t = s_i)$

> $ = \sum_{i=1}^K p(x_1, x_2, ..., x_t|z_t = s_i; \lambda) p(z_t = s_i) * p(x_{t+1}, z_{t+1} = s_k | z_t = s_i)$

> $ = \sum_{i=1}^K p(x_1, x_2, ..., x_t, z_t = s_i|\lambda) * p(x_{t+1}|z_{t+1} = s_k, z_t = s_i) p(z_{t+1} = s_k | z_t = s_i)$

> $ = \sum_{i=1}^K \alpha_{t,i} * p(x_{t+1}|z_{t+1} = s_k) a_{i,k}$

> $ = \left(\sum_{i=1}^K \alpha_{t,i} a_{i, k}\right) b_{k,x_{t+1}}$

> $p(x_1, x_2, ..., x_N|\lambda) = \sum_{i=1}^K \alpha_{N, i}$

+ backward

令$\beta_{t,k} = p(x_{t+1}, x_{t+2}, ..., x_{N} | z_{t} = s_k; \lambda)$，表示输入是$x_{t+1}, x_{t+2}, ..., x_N$，当前隐含状态是$k$，$z_{t}=s_k$，

初始化，
> $\beta_{N,k} = 1$

迭代，$t = N-1, N-2, ..., 1$，
> $\beta_{t, k} = p(x_{t+1},x_{t+2},...,x_N|z_t = s_k; \lambda)$

> $ = \sum_{i=1}^K p(x_{t+1}, x_{t+2}, ..., x_N, z_{t+1} = i | z_t = s_k; \lambda)$

> $ = \sum_{i=1}^K p(x_{t+2}, x_{t+3}, ..., x_N|x_{t+1}, z_t = s_k, z_{t+1} = i; \lambda)
* p(x_{t+1}, z_{t+1} = i|z_t = s_k; \lambda)$

> $ = \sum_{i=1}^K p(x_{t+2}, x_{t+3}, ..., x_N|z_{t+1} = i; \lambda) * p(x_{t+1}|z_{t+1} = i, z_t = s_k; \lambda) p (z_{t+1} = i|z_t = s_k; \lambda)$

> $ = \sum_{i=1}^K \beta_{t+1, i} * p(x_{t+1}|z_{t+1} = i; \lambda) a_{k, i}$ 

> $ = \sum_{i=1}^K \beta_{t+1, i} b_{i, x_{t+1}}  a_{k, i}$

终止条件，

> $ p(x_1, x_2, ..., x_N|\lambda) = $

> $ \sum_{i=1}^K p(x_1, x_2, x_3, ..., x_N, z_1 = s_i|\lambda) 
= \sum_{i=1}^K p(x_2, x_3, ..., x_N|x_1, z_1 = s_i; \lambda) p(x_1, z_1 = s_i|\lambda)$

> $ = \sum_{i=1}^K p(x_2, x_3, ..., x_N | z_1 = s_i; \lambda) p(x_1 | z_1 = s_i; \lambda) p(z_1 = s_i | \lambda)$

> $ = \sum_{i=1}^K \beta_{1,i} b_{i,x_1} \pi_{i}$

## 0.3 Decode问题求解（Viterbi）

计算最优状态序列：

> $Z^{*} = \{z_1^*, z_2^*, ..., z_N^*\} 
= \arg\max_{z_1, z_2, ..., z_N} p(z_1, z_2, ..., z_N | x_1, x_2, ..., x_N; \lambda )$

> $ = \arg\max_{z_1, z_2, ..., z_N} \frac{p(z_1, z_2, ..., z_N, x_1, x_2, ..., x_N)}
{p(x_1, x_2, ..., x_N)}$

> $= \arg\max_{z_1, z_2, ..., z_N} p(z_1, z_2, ..., z_N, x_1, x_2, ..., x_N)$


$\alpha, \beta$函数的由来：

> $p(z_n | X) = \frac{p(X | z_n)p(z_n)}{p(X)}$

> $ = \frac{p(x_1, x_2, ..., x_n, x_{n+1}, x_{n+2}, ..., x_N)|z_n) * p(z_n)} {p(X)}$

> $ = \frac{p(x_1, x_2, ..., x_n|z_n) p(x_{n+1}, x_{n+2}, ..., x_N|z_n) * p(z_n)} {p(X)}$

> $ = \frac{p(x_1, x_2, ..., x_n|z_n)p(z_n) * p(x_{n+1}, x_{n+2}, ..., x_N|z_n)} {p(X)}$

> $ = \frac{p(x_1, x_2, ..., x_n, z_n) * p(x_{n+1}, x_{n+2}, ..., x_N|z_n)} {p(X)}$

> $ = \frac{ \alpha_{n,z_n} * \beta_{n, z_n} } {p(X)}$

计算分母$p(X)$,

> $p(X) = \sum_{i=1}^K p(x_1, x_2, ..., x_n, z_n = s_i) * p(x_{n+1}, x_{n+2}, ..., x_N|z_n = s_i)$

> $ = \sum_{i=1}^K \alpha_{n, i} * \beta_{n, i}$

+ 定义经过路径$1, 2, ..., t$，且第$t$个状态为$k$的概率最大值为,

> $\delta_{t, k} = \max_{z_1, z_2, ..., z_{t-1}} p(z_1, z_2, ..., z_{t-1}, z_{t} = s_k, x_1, x_2, ..., x_{t}| \lambda), k = \{1, 2, ..., K\}$

初始状态，经过路径1，第1个隐含状态为$k$的概率(最大)值为，

> $\delta_{1, k} =  p(z_1 = s_k, x_1| \lambda)
= p(z_1 = s_k) * p(x_1 | z_1 = s_k; \lambda) = \pi_k * b_{k, x_1}$

经过路径$1, 2, ..., t+1$，且第$t$个状态为$k$的概率最大值为,

> $\delta_{t+1, k} = [\max_{z_1, z_2, ..., z_t} \delta_{t, i} * a_{i,k}] * b_{k,x_{t+1}}, 1 <= i <= K$

+ 定义路径$1, 2, ..., t$，在$t$步的隐含状态为$\wp_{t} = s_k$, 概率最大路径的第$t-1$个节点为,

> $\wp_{t, k} = \arg\max_{i} \delta_{t-1, i} a_{ik}, 1 <= i <= K$

> $\wp_{1, k} = 0$

### 0.3.0 初始状态

> $\delta_{1, k} =  p(z_1 = s_k, x_1| \lambda)
= p(z_1 = s_k) * p(x_1 | z_1 = s_k; \lambda) = \pi_k * b_{k, x_1}$

> $\wp_{1, k} = 0$

### 0.3.1 迭代

> $\delta_{t, k} = [\max \delta_{t-1, i} * a_{i,k}] * b_{k,x_t}$

> $\wp_{t, k} = \arg\max_{i} \delta_{t-1, i} a_{ik}$

### 0.3.2 终止条件

> $\delta_{N, k} = \max \delta_{N-1, i} * a_{i,k} * b_{k,x_N}$

> $\wp_{N, k} = \arg\max_{j} \delta_{N-1, j} a_{jk}$

> $P* = \max \delta_{N, k}, k = 1, 2, ..., K$

> $z_N^* = \arg\max_{k} \delta_{N, k}$

### 0.3.3 最优路径

> $z_N = \arg\max_{k} \delta_{N, k}$

$t = N-1, N-2, ..., 1$, 
> $z_t = \arg\max_{k} \wp_{t+1, z_{t+1}}, k = 1, 2, ..., K$

## 0.4 Learning问题求解 （EM）

> 观测数据: $X = \{x_1, x_2, ..., x_n\}$

> 隐含变量: $Z = \{z_1, z_2, ..., z_n\}$

> 参数: $\lambda = [\pi, A, B] = \arg\max_{\lambda} p(X | \theta)$。

参数估计首先想到极大似然估计；包含隐含变量的参数求解问题，非常适合使用EM算法。

### 0.4.0 E-步

考虑Q函数，计算完全数据的log似然关于隐变量的期望$\mathbb{E}_{Z}[\log p(X, Z|\theta)]$,
> $Q(\theta, \theta^{old}) = \sum_{Z} p(Z|\theta^{old}) \log p(X, Z|\theta)$

完全数据$(X, Z)$的似然函数为,

> $L(\theta | X, Z) = p(X, Z | \theta) = p(Z | \theta) p(X | Z, \theta)$ 

> $ = p(z_1) \left[ \prod_{n=2}^{N} p(z_n | z_{n-1}, \theta) \right] 
\left[ \prod_{n=1}^N p(x_n | z_n, \theta) \right]$

> $ = {\pi(k)}^{\gamma_{1, k}} \left[ \prod_{n=2}^{N} \prod_{i=1}^K \prod_{j=1}^K a_{i, j}^{\xi_{n, i, j}} \right] 
\left[ \prod_{n=1}^N \prod_{i=1}^K  b_{i, x_{n}}^{\gamma_{n, i}} \right]$

定义$\gamma_{n, i} = p(z_n = s_i)$, $\xi_{n, i, j} = p(z_{n-1} = s_i, z_n = s_j)$

完全数据log似然为, 

> $l(\theta|X, Z) = \log L(\theta|X, Z)$

> $ = \gamma_{1, k} \log \pi(k) + \sum_{n=2}^N \sum_{i=1}^K \sum_{j=1}^K \xi_{n, i, j} \log a_{i, j} + 
\sum_{n=1}^N \sum_{i=1}^K \gamma_{n, i} \log b_{i, x_n} $

完全数据log似然关于隐含变量的期望为，

> $\mathbb{E}[l(\theta|X, Z)] = \mathbb{E}[\gamma_{1, k}] \log \pi + \sum_{n=2}^N \sum_{i=1}^K \sum_{j=1}^K 
\mathbb{E}[\xi_{n, i, j}] 
\log a_{i, j} + 
\sum_{n=1}^N \sum_{i=1}^K 
\mathbb{E}[\gamma_{n, i}]
\log b_{i, x_n}$

需要计算$\mathbb{E}[\gamma_{n, i}]$以及$\mathbb{E}[\xi_{n, i, j}]$,

> $\mathbb{E}[\gamma_{n, i}] = \mathbb{E}[p(z_n = s_i)]$

借用0.3节的结论, 

> $p(z_n | X) = \frac{ \alpha_{n,z_n} * \beta_{n, z_n} }{p(X)}$

> $\mathbb{E}[\gamma_{n, i}] = \mathbb{E}[p(z_n = s_i| X)]$

> $p(z_{n-1}, z_n | X) = \frac{p(X | z_{n-1}, z_n) * p(z_{n-1}, z_n)} {p(X)} $

> $p(X | z_{n-1}, z_n) * p(z_{n-1}, z_n) = p(x1, x2, ..., x_{n-1}, x_n, x_{n+1}, ..., x_N | z_{n-1}, z_n) * p(z_{n-1}, z_n)$ 

> $ = p(x_1, x_2, ..., x_{n-1} | z_{n-1}, z_n) p(x_n | z_{n-1}, z_n) p(x_{n+1}, ..., x_N | z_{n-1}, z_n) * p(z_{n-1}, z_n)$

> $ = p(x_1, x_2, ..., x_{n-1} | z_{n-1}) p(x_n | z_n) p(x_{n+1}, ..., x_N | z_n) * p(z_{n-1}, z_n))$

> $ = \frac{p(x_1, x_2, ..., x_{n-1}, z_{n-1})}{p(z_{n-1})} b_{z_n, x_n} \beta{n, z_n} * p(z_n | z_{n-1}) p(z_{n-1})$

> $ = \alpha_{n-1, z_{n-1}} a_{z_{n-1}, z_n} b_{z_n, x_n} \beta_{n, z_n} $

> $p(z_{n-1}, z_n | X) = \frac{ \alpha_{n-1, z_{n-1}} a_{z_{n-1}, z_n} b_{z_n, x_n} \beta_{n, z_n} }{p(X)}$

> $\mathbb{E}[\xi_{n, i, j}] = \mathbb{E}[p(z_n = s_j | z_{n-1} = s_i)]$

### 0.4.1 M-步

分别计算$\mathbb{E}[l(\theta|X, Z)]$关于$\pi(k), a_{i, j}, b_{i, j}$的偏导, 并令其等于0,

考虑约束$\sum_{k=1}^K \pi(k) = 1$, $\sum_{j=1}^K a_{i, j} = 1$, $\sum_{j=1}^M b_{i, j} = 1$，考虑拉格朗日乘子。具体计算参考GMM，考虑对隐含变量每一个分量都计算偏导，然后求和利用约束条件求解计算得到拉格朗日乘子，然后带入得到隐含变量每一个分量的偏导值。

# 1. Conditional Random Field


## 1.0 定义
### 1.0.0 Markov Random Field

HMM是有向图模型，假设当前隐含状态只依赖前一个隐含状态，可能假设过于简单；为增加模型表达能力，可以使用无向图模型。

如果联合概率分布$p(Y)$满足局部、成对、全局马尔科夫性质，则称此概率分布为概率无向图模型，或者马尔科夫随机场Markov Random Field.

概率无向图模型最大特点就是易于因子分解，可以将联合概率分解为若干子联合概率（最大团）的乘积，从而易于计算（马尔科夫性）。概率无向图模型分解为最大团的乘积由Hammersley-Clifford定理保证。

> $p(Y) = \frac{1}{Z} \Pi_C \Psi_C (Y_C)$

> $Z = \Sigma_Y \Pi_C \Psi_C (Y_C)$


$\Psi_C (Y_C)$称作势函数；势函数严格为正，通常定义为指数函数，$\Psi_C (Y_C) = exp\{ - E(Y_C) \}$, 常表示为$k$个（特征）函数的和的指数函数。

> $\Psi_C (Y_C) = exp\{ \sum_{k} \theta_{C,k} f_{C,k}(Y_C) \}$

$Z$是归一化因子，被称为配分函数.


### 1.0.1 Conditional Random Field

有随机变量$X,Y$, 随机变量Y构成由无向图$G=(V,E)$表示的马尔科夫随机场, 如果对任意节点$v$,

> $p(Y_v | X, Y_{w \neq v}) = p(Y_v | X, Y_{w \sim v})$

称条件概率分布$p(Y|X)$为条件马尔科夫随机场。表示给定随机变量$X$, 节点$v$只依赖于其所有邻接点$w$, 与其它节点无关。一般$X$与$Y$的结构不一定要求相同。

+ 线性链条件随机场

$X = (X_1, X_2, ..., X_n), Y = (Y_1, Y_2, ..., Y_n)$均为线性链表示的随机变量序列.

如果给定$X$, $p(Y|X)$构成条件随机场, 则$p(Y|X)$为线性链条件随机场。

+ 与概率有向图模型对比

概率有向图模型常表示为$G(V, E)$;

概率无向图模型常表示为$G(V, F, E)$, 其中$F$是因子节点;



+ Logistic Regression

> $p(y|\boldsymbol{x}) = \frac{1}{Z(\boldsymbol{x})} 
exp \left\{ \lambda_y + \Sigma_{k=1}^K \lambda_{y, j} x_j \right\}$

> $Z(\boldsymbol{x}) = \sum_y exp \{ \lambda_y + \Sigma_{k=1}^K \lambda_{y, j} x_j \}$

令$f_{y', j}(y, \boldsymbol{x}) = \mathbb{1}(y' = y)x_j$表示特征权重, $f_{y'}(y, \boldsymbol{x}) = \mathbb{1}(y' = y )$表示偏置权重; 用$f_k$表示特征, $\lambda_k$表示对应权重,

> $p(y | \boldsymbol{x}) = \frac{1}{Z(\boldsymbol{x})}
exp \left\{ \sum_{k=1}^K \lambda_k f_k(y, \boldsymbol{x}) \right\}$
